In [2]:
import sys
from PyQt5 import uic, QtWidgets, QtCore, QtGui
from PyQt5.QtWidgets import QApplication, QDialog, QMainWindow, QPushButton
from PyQt5.QtCore import Qt, QRunnable, pyqtSlot, QThreadPool, QObject, pyqtSignal
import pandas as pd
import random
import import_ipynb
from Redapy_query import query_final
from Scrapeo_2017 import make_query, make_dataframe
import traceback, sys
import redapy
import os

QtWidgets.QApplication.setAttribute(QtCore.Qt.AA_EnableHighDpiScaling, True) #enable highdpi scaling
QtWidgets.QApplication.setAttribute(QtCore.Qt.AA_UseHighDpiPixmaps, True) #use highdpi icons

class Redapy_GUI(QMainWindow):
    
    def __init__(self):
        super().__init__()
        uic.loadUi("Redapy.ui", self)
        
        self.threadpool = QThreadPool()
        print("Multithreading with maximum %d threads" % self.threadpool.maxThreadCount())
        
        # Desactiva Var2 cuando consulta es Frequency
        self.comboBox_consulta.currentIndexChanged.connect(self.on_change)
                        
        # Cargar datos cuando se selecciona BD
        self.comboBox_BD.currentTextChanged.connect(self.cargar_datos)
        self.comboBox_var1.setEditable(True)
        self.comboBox_var1.setInsertPolicy(QtWidgets.QComboBox.NoInsert)
        self.comboBox_var1.completer().setCompletionMode(QtWidgets.QCompleter.PopupCompletion)
        self.comboBox_var2.setEditable(True)
        self.comboBox_var2.setInsertPolicy(QtWidgets.QComboBox.NoInsert)
        self.comboBox_var2.completer().setCompletionMode(QtWidgets.QCompleter.PopupCompletion)
        self.comboBox_filtro.setEditable(True)
        self.comboBox_filtro.setInsertPolicy(QtWidgets.QComboBox.NoInsert)
        self.comboBox_filtro.completer().setCompletionMode(QtWidgets.QCompleter.PopupCompletion)
        self.comboBox_seleccion_ubigeos.setEditable(True)
        self.comboBox_seleccion_ubigeos.setInsertPolicy(QtWidgets.QComboBox.NoInsert)
        self.comboBox_seleccion_ubigeos.completer().setCompletionMode(QtWidgets.QCompleter.PopupCompletion)

        # boton Randomize
        self.Boton_Randomize.clicked.connect(self.randomize)
        
        # boton Limpiar
        self.Limpiar_Inputs.clicked.connect(self.reset_main_ui)
        
        # boton lineas_redatam
        self.Boton_Lineas_Redatam.clicked.connect(self.lineas_redatam)
        
        # boton Scrapeo
        self.Boton_Scrapear.clicked.connect(self.consulta)
   
    def consulta(self):
        # Pass the function to execute
        worker = Worker(self.procesar) # Any other args, kwargs are passed to the run function
        worker.signals.result.connect(self.print_output)
        worker.signals.finished.connect(self.thread_complete)
        worker.signals.progress.connect(self.progress_fn)

        # Execute
        self.threadpool.start(worker)
        
    def print_output(self, s):
        print(s)
        
    def thread_complete(self):
        print("PROCESO FINALIZADO")
        
    def progress_fn(self, n):
        print("%d%% PROCESADO" % n)
        
    def procesar(self, progress_callback):
        df_query = make_query(self.Lineas_Redatam.toPlainText(),BASE_URL)
        df = pd.concat(make_dataframe(df_query))
        model = pandasModel(df)
        self.tabla_resultados.setModel(model)
        self.tabla_resultados.show()
        print("Iniciando limpieza...")
        
        if self.comboBox_consulta.currentText() == "Crosstab":
            try:
                df = redapy.tabla_cruzada(df)
                model = pandasModel(df)
                self.tabla_resultados.setModel(model)
                self.tabla_resultados.show()
                print("Limpieza exitosa")
            except:
                print("Error limpiando df")
        if self.comboBox_consulta.currentText() == "Frequency":
            try: 
                df = redapy.conversion_redatam_matriz(df)
                model = pandasModel(df)
                self.tabla_resultados.setModel(model)
                self.tabla_resultados.show()
                print("Limpieza exitosa")
            except:
                print("Error limpiando df")
            
        
    def on_change(self, newIndex):
        if newIndex==2:
            self.comboBox_var2.setEnabled(False)
            self.comboBox_var2.setCurrentIndex(0)
            self.comboBox_nivel_var2.setEnabled(False)
            self.comboBox_nivel_var2.setCurrentIndex(0)
        else: 
            self.comboBox_var2.setEnabled(True)
            self.comboBox_nivel_var2.setEnabled(True)
                 
    def cargar_datos(self, value):
        if self.comboBox_BD.currentText():
            datos_bd = self.comboBox_BD.currentText()
#             datos_bd_ubigeos = pd.read_excel(datos_bd+'.xlsx',header=0, sheet_name=0, converters={'UBIGEO':str, 'NOMBRE':str})
#             datos_bd_variables = pd.read_excel(datos_bd+'.xlsx',header=0, sheet_name=1, converters={'VAR':str, 'CATEGORIA':str})
#             global BASE_URL
#             datos_bd_url = pd.read_excel(datos_bd+'.xlsx',header=0, sheet_name=2, converters={'URL':str})
            
            datos_bd_ubigeos = pd.read_csv(datos_bd+'//UBIGEOS.csv', sep=";", converters={'UBIGEO':str, 'NOMBRE':str})
            datos_bd_variables = pd.read_csv(datos_bd+'//VAR.csv', sep=";", converters={'VAR':str, 'CATEGORIA':str})
            global BASE_URL
            datos_bd_url = pd.read_csv(datos_bd+'//URL.csv', sep=";", converters={'URL':str})
            BASE_URL = datos_bd_url['URL'][0]
            
            lista_ubigeos = list(set(datos_bd_ubigeos['UBIGEO'].tolist()))
            self.comboBox_seleccion_ubigeos.clear()
            self.comboBox_seleccion_ubigeos.addItems(lista_ubigeos)
        
            lista_variables = list(set(datos_bd_variables['VAR_COMPLETO'].tolist()))
            self.comboBox_filtro.clear()
            self.comboBox_filtro.addItems(lista_variables)
            self.comboBox_var1.clear()
            self.comboBox_var1.addItems(lista_variables)
            self.comboBox_var2.clear()
            self.comboBox_var2.addItems(lista_variables)
            global comboBox_consulta_max
            comboBox_consulta_max = self.comboBox_consulta.count()
            global comboBox_var1_max
            comboBox_var1_max = self.comboBox_var1.count()
            global comboBox_var2_max
            comboBox_var2_max = self.comboBox_var2.count()
            global comboBox_filtro_max
            comboBox_filtro_max = self.comboBox_filtro.count()
            global comboBox_seleccion_ubigeos 
            comboBox_seleccion_ubigeos = self.comboBox_seleccion_ubigeos.count()
        
    def randomize (self, value):
        self.comboBox_var1.setCurrentIndex(random.randint(0,comboBox_var1_max))
        self.comboBox_var2.setCurrentIndex(random.randint(0,comboBox_var2_max))
        self.comboBox_filtro.setCurrentIndex(random.randint(0,comboBox_filtro_max))
        self.comboBox_seleccion_ubigeos.setCurrentIndex(random.randint(0,comboBox_seleccion_ubigeos))
        self.comboBox_consulta.setCurrentIndex(random.randint(1,2))
        self.comboBox_var2.update()
        self.comboBox_nivel_var2.update()

    def reset_main_ui(self, value):
        self.comboBox_consulta.setCurrentIndex(0)
        self.comboBox_var1.setCurrentText("")
        self.comboBox_nivel_var1.setCurrentIndex(0)
        self.comboBox_var2.setCurrentText("")
        self.comboBox_nivel_var2.setCurrentIndex(0)
        self.comboBox_filtro.setCurrentText("")
        self.comboBox_nivel_filtro.setCurrentIndex(0)
        self.comboBox_seleccion_ubigeos.setCurrentText("")
        self.comboBox_seleccion_nivel.setCurrentIndex(0)
        self.comboBox_areabreak_nivel.setCurrentIndex(0)
        
    def lineas_redatam(self, value):
        tipo, variables1, variables2, selection, area_break = ([] for i in range(5))
        tipo = self.comboBox_consulta.currentText() if self.comboBox_consulta.currentText() else ""
        variables1.append(self.comboBox_var1.currentText()) if self.comboBox_var1.currentText() else ""
        variables2.append(self.comboBox_var2.currentText()) if self.comboBox_var2.currentText() else ""
        selection.append(self.comboBox_seleccion_ubigeos.currentText()) if self.comboBox_seleccion_ubigeos.currentText() else ""
        if selection:
            if len(selection[0]) == 2:
                selection[0] = "Departam "+selection[0]
            if len(selection[0]) == 4:
                selection[0] = "Provinci "+selection[0]
            if len(selection[0]) == 6:
                selection[0] = "Distrito "+selection[0]
        area_break.append(self.comboBox_areabreak_nivel.currentText()) if self.comboBox_areabreak_nivel.currentText() else ""
        universe_filter = "" 
        title = ""
        self.Lineas_Redatam.setText(query_final(tipo=tipo,variables1=variables1,variables2=variables2,selection=selection,area_break=area_break))
                
class WorkerSignals(QObject):
    finished = pyqtSignal()
    error = pyqtSignal(tuple)
    result = pyqtSignal(object)
    progress = pyqtSignal(int)
    
class Worker(QRunnable):
    def __init__(self, fn, *args, **kwargs):
        super(Worker, self).__init__()
        # Store constructor arguments (re-used for processing)
        self.fn = fn
        self.args = args
        self.kwargs = kwargs
        self.signals = WorkerSignals()

        # Add the callback to our kwargs
        self.kwargs['progress_callback'] = self.signals.progress

    @pyqtSlot()
    
    def run(self):
        try:
            result = self.fn(*self.args, **self.kwargs)
        except:
            traceback.print_exc()
            exctype, value = sys.exc_info()[:2]
            self.signals.error.emit((exctype, value, traceback.format_exc()))
        else:
            self.signals.result.emit(result)  # Return the result of the processing
        finally:
            self.signals.finished.emit()  # Done

class pandasModel(QtCore.QAbstractTableModel):

    def __init__(self, data):
        QtCore.QAbstractTableModel.__init__(self)
        self._data = data

    def rowCount(self, parent=None):
        return self._data.shape[0]

    def columnCount(self, parnet=None):
        return self._data.shape[1]

    def data(self, index, role=Qt.DisplayRole):
        if index.isValid():
            if role == Qt.DisplayRole:
                return str(self._data.iloc[index.row(), index.column()])
        return None

    def headerData(self, col, orientation, role):
        if orientation == Qt.Horizontal and role == Qt.DisplayRole:
            return self._data.columns[col]
        return None

if __name__=='__main__':
    app = QtWidgets.QApplication(sys.argv)
    if app is None:
        app = QtGui.QApplication(sys.argv)
    GUI = Redapy_GUI()
    GUI.show()
    # sys.exit(app.exec_())
    app.exec_()
    del app

Multithreading with maximum 12 threads
Scrapeo iniciado
Se cargó página REDATAM con éxito
Iniciando scrapeo...
La tabla cargó correctamente
Tabla scrapeada con éxito en:
0:00:52.549570
Iniciando limpieza...
Error limpiando df
None
PROCESO FINALIZADO
Scrapeo iniciado
Se cargó página REDATAM con éxito
Iniciando scrapeo...
La tabla cargó correctamente
Tabla scrapeada con éxito en:
0:01:56.518050
Iniciando limpieza...
Limpieza exitosa
None
PROCESO FINALIZADO
